# Data Science template 

In [2]:
#conda update -n base -c defaults conda

In [1]:
# Importing libraries and packages

from pydoc import help  

import datetime as dt
import pandas as pd
import numpy as np
import os
import re
from pylab import rcParams

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from IPython.display import display, HTML

from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error
import xgboost as xgb

import itertools
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV   
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings('ignore')


### Import your data and start exploring

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
# Read csv file into pandas dataframe

df = pd.read_csv("PutYourFileNameHere.csv")

In [ ]:
df = df.round(2)

In [ ]:
# get dataset info

print("############################################")
print("          Info Of the Data Set")
print("############################################")
df.info()

In [ ]:
# exploring distribution

fig, ax = plt.subplots(6, 2, figsize = (15, 13))
sns.boxplot(x= df["var"], ax = ax[0,0])
sns.distplot(df['var'], ax = ax[0,1])
sns.boxplot(x= df["var"], ax = ax[1,0])
sns.distplot(df['var'], ax = ax[1,1])
sns.boxplot(x= df["var"], ax = ax[2,0])
sns.distplot(df['var'], ax = ax[2,1])
sns.boxplot(x= df["var"], ax = ax[3,0])
sns.distplot(df['var'], ax = ax[3,1])
sns.boxplot(x= df["var"], ax = ax[4,0])
sns.distplot(df['var'], ax = ax[4,1])
sns.boxplot(x= df["var"], ax = ax[5,0])
sns.distplot(df['var'], ax = ax[5,1])
plt.tight_layout()

In [ ]:
# Create a function that we can re-use

def show_distribution(var_data):
    from matplotlib import pyplot as plt

    # Get statistics
    min_val = var_data.min()
    max_val = var_data.max()
    mean_val = var_data.mean()
    med_val = var_data.median()
    mod_val = var_data.mode()[0]

    print('Minimum:{:.2f}\nMean:{:.2f}\nMedian:{:.2f}\nMode:{:.2f}\nMaximum:{:.2f}\n'.format(min_val,
                                                                                            mean_val,
                                                                                            med_val,
                                                                                            mod_val,
                                                                                            max_val))

    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize = (10,4))

    # Plot the histogram   
    ax[0].hist(var_data)
    ax[0].set_ylabel('Frequency')

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mean_val, color = 'cyan', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=med_val, color = 'red', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mod_val, color = 'yellow', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=max_val, color = 'gray', linestyle='dashed', linewidth = 2)

    # Plot the boxplot   
    ax[1].boxplot(var_data, vert=False)
    ax[1].set_xlabel('Value')

    # Add a title to the Figure
    fig.suptitle('Data Distribution')

    # Show the figure
    fig.show()


# Get the variable to examine
col = df['VarName']
# Call the function
show_distribution(col)

In [ ]:
# heat map

plt.figure(figsize=(10,6))
sns.heatmap(df.corr(),cmap=plt.cm.Reds,annot=True)
plt.title('Heatmap displaying the relationship between the features of the data',
         fontsize=13)
plt.show()

In [ ]:
# Exploring relationships - this code will create scatter plots 

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
  
# selecting numerical features
features = ['Var1', 'Var2', 'Var3', 'Var4']
   
# plotting the scatter matrix with the features
scatter_matrix(df[features], figsize=(12,12))
plt.xticks(rotation=90)
plt.show()

In [ ]:
# If you want to compare groups based on one or more quantitative variables, use this code

col_list = ['GroupingVarName', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6']
short_df = df[col_list]

import seaborn as sns

rs=1999
 
df_long = pd.melt(short_df.sample(1000,random_state=rs), "GroupingVarName", var_name="Columns", value_name="Values")   
f,ax = plt.subplots(figsize=(16,8))
#plt.xticks(rotation=90)
#plt.ylim(0, 10)
#plt.xlim(0, None)
sns.boxplot(x="Columns", y="Values", hue="GroupingVarName", data=df_long)

In [ ]:
df1 = df.copy()
df1['Name 1'] = df['var1']
df1['Name 2'] = df['var2']
df1['Name 3'] = df['var3']
df1['Name'] = df['CatVar']
boxplot = df1.boxplot(column=['Name 1', 'Name 2', 'Name 3'], by=['Name'], rot=45, fontsize=10, figsize=(20,10))

In [ ]:
# missing value handling

meanx = df['var'].mean()
df['var'] = df['var'].fillna(meanx)

In [ ]:
# normality check

print("Skewness: %f" %df['var'].skew())
print("Kurtosis: %f" %df['var'].kurt())

### Data transformations

In [ ]:
# Removing outliers - Use if you want to get rid of outliers and rerun the matrix
# You can add more lines for additional variables
# Typically we use 3 standard deviations (e.g., Six Sigma - 3 SDs above and below mean)

#Create a new dataset (df2) so that you don't replace the values in df
df2=copy.df()

from scipy import stats
df2=df2[(np.abs(stats.zscore(df2['Var1'])) < 3)]
df2=df2[(np.abs(stats.zscore(df2['Var2'])) < 3)]
df2=df2[(np.abs(stats.zscore(df2['Var3'])) < 3)]
df2=df2[(np.abs(stats.zscore(df2['Var4'])) < 3)]


In [ ]:
# Use this if doing normalization - setting the scale to be between 0 and 1
# You would only do this if you want to standardize your variables and have all of them on same scale
# This would be preferred over creating z scores if your distribution is not normal

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Again, creating a new dataset so we don't replace the values in the original dataset
df2=df.copy()

# Applying scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['Var1', 'Var2', 'Var3', 'Var4']
df2[num_vars] = scaler.fit_transform(df2[num_vars])

df2

In [ ]:
# Use this if doing standardization (z scores) - you would do this or the one above, but not both
# This code creates a z score that has zero as the mean. It is different than normalization

df['Var1'] = (df['Var1'] - df['Var1'].mean())/df['Var1'].std(ddof=0)

In [ ]:
# recode categorical variables

df['var'] = df['var'].replace({1: 'name', 2: 'name', 3: 'name'})
df.head()

In [ ]:
# Creating a dummy variable

df = pd.concat([df,pd.get_dummies(df['var'], prefix=None)],axis=1)

In [ ]:
# Run this code if you get errors about infinity values or missing values and rerun time series
# Note that this can cause you to end up with no values so you may need to try to transform, etc. instead of simply deleting

# Replace Inf values with NaN
df = df.replace([np.inf, -np.inf], np.nan)
# Drop all occurences of NaN
df = df.dropna()
# Double check these are all gone
df.isnull().any().any()

### Seasonal decomposition

In [ ]:
# Decomposition model to get seasonality, trend and noise
# This code creates a time series with 3 components (seasonal, trend, and noise)
# Change Var1 with your variable of interest. Change df to your dataset name if it is different

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

result=seasonal_decompose(df['Var1'], model='additive', freq=12, extrapolate_trend=12)


result.plot()
# pyplot.show()


In [ ]:
# Run this code to save the three components so you can download to csv file. 

df['trend'] = result.trend

df['residual'] = result.resid

df['seasonal'] = result.seasonal

df.head()

In [ ]:
# Use this code to save your dataset to a csv file that will load into this folder

df.to_csv('dataset.csv')

### Multiple linear regression

In [ ]:
# This is code to run a linear regression analysis
# Change df to be your dataset name if it is different
# Change Target to be your dependent variable
# Change Var1, Var2, Var3, Var4 to be the name of your predictor variables
# You can add more predictors if you want

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std


new_model = ols("""Target ~ Var1 
                                     + Var2 
                                     + Var3
                                     + Var4""", data=df).fit()
# summarize our model
new_model_summary = new_model.summary()
HTML(new_model_summary.as_html())

In [ ]:
 #This code block runs regression diagnostic visualizations (partial plots)

fig = plt.figure(figsize=(20,12))
fig = sm.graphics.plot_partregress_grid(new_model, fig=fig)

In [ ]:
# This produces our four regression plots for specified feature variable (e.g., one of your predictor variables)
# You would run this code for each predictor variable

fig = plt.figure(figsize=(15,8))

# pass in the model as the first parameter, then specify the 
# predictor variable we want to analyze
fig = sm.graphics.plot_regress_exog(new_model, "Var1", fig=fig)

### Decision trees 

In [ ]:
# Creating a list of features for the decision tree
# Include in this list, all the predictor variables that you want to use in your decision tree

features=['Var1',
 'Var2',
 'Var3',
 'Var4',
 'Var5',
 'Var6']

In [ ]:
target = ['var']

In [ ]:
df = df[df['var'].notna()]
df

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import datasets, tree

y = size[target]
X = size[features]#.astype(float)
dt = DecisionTreeRegressor(min_samples_split=100, min_samples_leaf=100, random_state=99, max_depth=10, max_leaf_nodes=10)
dt.fit(X, y)

In [ ]:
text_representation = tree.export_text(dt)
print(text_representation)

In [ ]:
with open("decistion_tree.log", "w") as fout:
    fout.write(text_representation)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt,
                   feature_names=features,
                   class_names=target,
                   filled=True)

### LazyPredict Classifer - image

In [ ]:
# Classification Example

from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

data = df
X = data.data
y= data.target

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.5,random_state =123)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

### XG Boost

In [ ]:
#remove date variables (correlation b/w region and store will be addressed in differrent models )
df.drop(['var1','var2','var3','var4'], axis=1, inplace=True) 

#train test split
train_size = int(len(df1.index) * 0.9)
train, test = df1[0:train_size], df1[train_size:len(df1.index)]
print(len(df1.index))
print(len(train))
print(len(test))


In [ ]:
# # Features Scaling
# train = (train - train.mean()) / train.std()
# test = (test - test.mean()) / test.std()

X_train, y_train = train.drop(columns=["TargetVar"], axis=1), train["TargetVar"]
X_test, y_test = test.drop(columns=["TargetVar"], axis=1), test["TargetVar"]

In [ ]:
matrix_train = xgb.DMatrix(X_train, label = y_train)
matrix_test = xgb.DMatrix(X_test, label = y_test)

# Run XGB 
model = xgb.train(params={'objective':'reg:squarederror','eval_metric':'mae'}
                ,dtrain = matrix_train, num_boost_round = 500, 
                early_stopping_rounds = 20, evals = [(matrix_test,'test')],)

In [ ]:
print(model.best_score)
print(model.best_iteration)
print("Best tree limit:", model.best_ntree_limit)

y_pred = model.predict(matrix_test, model.best_ntree_limit)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
xgb.plot_importance(model, height=0.5, ax=ax);

In [ ]:
import random
df_pred = test.copy()
df_pred['Forecast'] = y_pred
# Plot Actual vs. Forecast for a given store 
sample = random.choice(df_pred.store_id.unique())

plt.figure(figsize=(16, 6))
plt.plot(df_pred[df_pred.store_id == sample]['TargetVar'],label='Actual')
plt.plot(df_pred[df_pred.store_id == sample]['Forecast'], 
         color='red', label='Forecast')
plt.legend();

### KMeans - use K prototype for mixed data

In [ ]:
# import KMeans
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt

KMeans(init='random', n_clusters=10, n_init=10, random_state=1)

In [ ]:
#initialize kmeans parameters
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

#create list to hold SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(df)
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

### PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(df)
X_scaled[:5]

In [ ]:
features = X_scaled.T
cov_matrix = np.cov(features)
cov_matrix[:5]

In [ ]:
cov_matrix2 = np.nan_to_num(cov_matrix, copy=True, nan=0.0, posinf=30, neginf=-30)

In [ ]:
values, vectors = np.linalg.eig(cov_matrix2)
values[:5]

In [ ]:
projected_1 = X_scaled.dot(vectors.T[0])
projected_2 = X_scaled.dot(vectors.T[1])
projected_3 = X_scaled.dot(vectors.T[2])
y = df['CatVar1']
z = df['CatVar2']
res = pd.DataFrame(projected_1, columns=['PC1'])
res['PC2'] = projected_2
res['PC3'] = projected_3
res['CatVar1'] = y
res['CatVar2'] = z
res.head()

In [ ]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=res, x="PC1", y="PC2", hue="CatVar1")

In [ ]:
plt.figure(figsize=(20, 10))
sns.scatterplot(data=res, x="PC1", y="PC2", hue="CatVar2")

### Logistic regression

In [ ]:
train=pd.read_csv('df.csv')
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.drop('TargetVar',axis=1), 
                                                    train['TargetVar'], test_size=0.30, 
                                                    random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))